In [319]:
import math
import numpy as np
from scipy.special import gamma
import scipy.stats as statistics
import pandas as pd
import swifter
from typing import Literal
import pickle

In [2]:
with open('results/data_gd_cut.p', 'rb') as f:
    data = pickle.load(f)

In [3]:
with open("results/stats_gd_cut.p",'rb') as file:
    stats = pickle.load(file)

In [4]:
data

instance  \
n p i                                             
3 1 0  <MaxCut.MaxCut object at 0x7fcbc7407910>   
    1  <MaxCut.MaxCut object at 0x7fcb52d05540>   
  2 0  <MaxCut.MaxCut object at 0x7fcbc7407910>   
    1  <MaxCut.MaxCut object at 0x7fcb52d05540>   
4 1 0  <MaxCut.MaxCut object at 0x7fcb52d05720>   
...                                         ...   
5 4 5  <MaxCut.MaxCut object at 0x7fcb52d07430>   
    6  <MaxCut.MaxCut object at 0x7fcb52d07730>   
    7  <MaxCut.MaxCut object at 0x7fcb52d07a30>   
    8  <MaxCut.MaxCut object at 0x7fcb52d07d30>   
    9  <MaxCut.MaxCut object at 0x7fcb52af4070>   

                                                    tdvp  \
n p i                                                      
3 1 0  instance                                    <M...   
    1  instance                                    <M...   
  2 0  instance                                    <M...   
    1  instance                                    <M...   
4 1 0  instance                                    <M...   
...                                                  ...   
5 4 5  instance                                    <M...   
    6  instance                                    <M...   
    7  instance                                    <M...   
    8  instance                                    <M...   
    9  instance                                    <M...   

                                                   scipy  \
n p i                                                      
3 1 0  instance                                    <M...   
    1  instance                                    <M...   
  2 0  instance                                    <M...   
    1  instance                                    <M...   
4 1 0  instance                                    <M...   
...                                                  ...   
5 4 5  instance                                    <M...   
    6  instance                                    <M...   
    7  instance                                    <M...   
    8  instance                                    <M...   
    9  instance                                    <M...   

                                        gradient_descent  
n p i                                                     
3 1 0  algorithm                                     ...  
    1  algorithm                                     ...  
  2 0  algorithm                                     ...  
    1  algorithm                                     ...  
4 1 0  algorithm                                     ...  
...                                                  ...  
5 4 5  algorithm                                     ...  
    6  algorithm                                     ...  
    7  algorithm                                     ...  
    8  algorithm                                     ...  
    9  algorithm                                     ...  

[74 rows x 4 columns]

In [5]:
data_col = pd.DataFrame(index=data['tdvp'].xs((4,5), level=[0,1]).index, columns=data['tdvp'].loc[4,5,0].index)
for i in data_col.index:
    data_col.loc[i] = data['tdvp'].xs((4,5), level=[0,1]).loc[i]
data_col[data_col['approximation ratio'].apply(lambda x: np.isnan(x))]

,instance,p,n,delta_0,tollarance,algorithm,res,delta,value,path,...,answer,bin_answer,best_cost_value,approximation ratio,expectation approximation ratio,circuit count,gate count,path length,groundspace overlap,groundspace sharpness
i,,,,,,,,,,,,,,,,,,,,,
0,<MaxCut.MaxCut object at 0x7fcb52d05720>,5,4,"(1, 1, 1, 1, 1, 1, 1, 1, 1, 1)",0.01,tdvp,\n None terminated with no sucess with ...,"(1, 1, 1, 1, 1, 1, 1, 1, 1, 1)",None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,<MaxCut.MaxCut object at 0x7fcb52d05d50>,5,4,"(1, 1, 1, 1, 1, 1, 1, 1, 1, 1)",0.01,tdvp,\n None terminated with no sucess with ...,"(1, 1, 1, 1, 1, 1, 1, 1, 1, 1)",None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,<MaxCut.MaxCut object at 0x7fcb52d06290>,5,4,"(1, 1, 1, 1, 1, 1, 1, 1, 1, 1)",0.01,tdvp,\n None terminated with no sucess with ...,"(1, 1, 1, 1, 1, 1, 1, 1, 1, 1)",None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
relevant_count = pd.DataFrame(index=stats.index, columns=pd.MultiIndex.from_product([['relevance'],['count', 'total', 'ratio']]))

def count_relevance(col):
    n,alg,p = idx = col.name
    df = data[alg].xs((n,p), level=[0,1])
    data_col = pd.DataFrame(index=df.index, columns=data[alg].loc[n,p,0].index)
    for i in data_col.index:
        data_col.loc[i] = df.loc[i]
    col['count'] = len(data_col[data_col['answer'].apply(lambda x: not np.isnan(x))])
    col['total'] = len(data_col)
    col['ratio'] = col['count'] / col['total']



In [16]:
relevant_count['relevance'].apply(count_relevance, axis=1)
relevant_count

relevance                
                         count total     ratio
n Algorithm        p                          
3 tdvp             1         2     2       1.0
                   2         2     2       1.0
  scipy            1         2     2       1.0
                   2         2     2       1.0
  gradient_descent 1         2     2       1.0
                   2         2     2       1.0
4 tdvp             1         6     6       1.0
                   2         6     6       1.0
                   3         5     6  0.833333
                   4         4     6  0.666667
                   5         3     6       0.5
  scipy            1         6     6       1.0
                   2         6     6       1.0
                   3         6     6       1.0
                   4         6     6       1.0
                   5         6     6       1.0
  gradient_descent 1         6     6       1.0
                   2         6     6       1.0
                   3         6     6       1.0
                   4         6     6       1.0
                   5         6     6       1.0
5 tdvp             1        10    10       1.0
                   2         9    10       0.9
                   3         7    10       0.7
                   4         6    10       0.6
  scipy            1        10    10       1.0
                   2        10    10       1.0
                   3        10    10       1.0
                   4        10    10       1.0
  gradient_descent 1        10    10       1.0
                   2        10    10       1.0
                   3        10    10       1.0
                   4        10    10       1.0

In [17]:
stats = stats.drop(columns=['relevance']).join(relevant_count)
stats.head(5)

/tmp/ipykernel_94938/2346214742.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  stats = stats.drop(columns=['relevance']).join(relevant_count)


expectation approximation ratio                      \
                                                mean       std       min   
n Algorithm        p                                                       
3 tdvp             1                        0.911088  0.073714  0.858965   
                   2                        0.930390  0.063845  0.885245   
  scipy            1                        0.778976  0.112656  0.699316   
                   2                        0.930466  0.058679  0.888974   
  gradient_descent 1                        0.306478  0.432672  0.000533   

                               groundspace overlap                      \
                           max                mean       std       min   
n Algorithm        p                                                     
3 tdvp             1  0.963212            0.869012  0.145009  0.766475   
                   2  0.975536            0.931829  0.071453  0.881304   
  scipy            1  0.858635            0.766332  0.006235  0.761923   
                   2  0.971958            0.893380  0.117401  0.810365   
  gradient_descent 1  0.612424            0.358776  0.307027  0.141675   

                               groundspace sharpness            ...  steps  \
                           max                  mean       std  ...    max   
n Algorithm        p                                            ...          
3 tdvp             1  0.971549              0.294665  0.181596  ...   87.0   
                   2  0.982354              0.227546  0.135601  ...  147.0   
  scipy            1  0.770741              0.423131  0.003925  ...   24.0   
                   2  0.976395              0.271914  0.169845  ...   50.0   
  gradient_descent 1  0.575877              0.421463  0.102879  ...   59.0   

                     path length                                \
                            mean       std       min       max   
n Algorithm        p                                             
3 tdvp             1    0.912754  0.036438  0.886988  0.938520   
                   2    0.823661  0.014654  0.813299  0.834024   
  scipy            1    0.131371  0.094648  0.064445  0.198298   
                   2    0.081390  0.084589  0.021576  0.141203   
  gradient_descent 1    0.230496  0.310428  0.010990  0.450001   

                     expectation approximation ratio per circuits            \
                                                             mean       std   
n Algorithm        p                                                          
3 tdvp             1                                     0.000309 -0.000002   
                   2                                     0.000069 -0.000013   
  scipy            1                                     0.031795  0.003681   
                   2                                     0.018609  0.000647   
  gradient_descent 1                                     0.001293  0.001826   

                     relevance              
                         count total ratio  
n Algorithm        p                        
3 tdvp             1         2     2   1.0  
                   2         2     2   1.0  
  scipy            1         2     2   1.0  
                   2         2     2   1.0  
  gradient_descent 1         2     2   1.0  

[5 rows x 41 columns]

In [18]:
with open("results/stats_gd_cut.p",'wb') as file:
    pickle.dump(stats,file)

In [ ]:
stats.head(10)

expectation approximation ratio                      \
                                                mean       std       min   
n Algorithm        p                                                       
3 tdvp             1                        0.911088  0.073714  0.858965   
                   2                        0.930390  0.063845  0.885245   
  scipy            1                        0.778976  0.112656  0.699316   
                   2                        0.930466  0.058679  0.888974   
  gradient_descent 1                        0.306478  0.432672  0.000533   
                   2                        0.652124  0.047679  0.618410   
4 tdvp             1                        0.555697  0.320149  0.133846   
                   2                        0.914538  0.023949  0.873142   
                   3                        0.926812  0.027653  0.895857   
                   4                        0.943432  0.049538  0.869999   

                               groundspace overlap                      \
                           max                mean       std       min   
n Algorithm        p                                                     
3 tdvp             1  0.963212            0.869012  0.145009  0.766475   
                   2  0.975536            0.931829  0.071453  0.881304   
  scipy            1  0.858635            0.766332  0.006235  0.761923   
                   2  0.971958            0.893380  0.117401  0.810365   
  gradient_descent 1  0.612424            0.358776  0.307027  0.141675   
                   2  0.685838            0.636691  0.049783  0.601489   
4 tdvp             1  0.963225            0.464344  0.373130  0.019871   
                   2  0.947408            0.895209  0.049664  0.831052   
                   3  0.966206            0.866436  0.068250  0.795922   
                   4  0.974706            0.930693  0.062414  0.843536   

                               groundspace sharpness            ...   steps  \
                           max                  mean       std  ...     max   
n Algorithm        p                                            ...           
3 tdvp             1  0.971549              0.294665  0.181596  ...    87.0   
                   2  0.982354              0.227546  0.135601  ...   147.0   
  scipy            1  0.770741              0.423131  0.003925  ...    24.0   
                   2  0.976395              0.271914  0.169845  ...    50.0   
  gradient_descent 1  0.575877              0.421463  0.102879  ...    59.0   
                   2  0.671892              0.479558  0.014190  ...    95.0   
4 tdvp             1  0.962394              0.331865  0.164583  ...    82.0   
                   2  0.943499              0.297600  0.061860  ...   701.0   
                   3  0.967577              0.324697  0.090627  ...  3000.0   
                   4  0.977556              0.232359  0.100608  ...   225.0   

                     path length                                \
                            mean       std       min       max   
n Algorithm        p                                             
3 tdvp             1    0.912754  0.036438  0.886988  0.938520   
                   2    0.823661  0.014654  0.813299  0.834024   
  scipy            1    0.131371  0.094648  0.064445  0.198298   
                   2    0.081390  0.084589  0.021576  0.141203   
  gradient_descent 1    0.230496  0.310428  0.010990  0.450001   
                   2    0.088509  0.090446  0.024554  0.152464   
4 tdvp             1    0.837968  0.199199  0.434681  0.955152   
                   2    0.782392  0.116077  0.602306  0.896681   
                   3    0.593010  0.182262  0.372322  0.862888   
                   4    0.675338  0.099432  0.564874  0.806639   

                     expectation approximation ratio per circuits            \
                                                             mean       std   
n Algorithm        p    

In [ ]:
stats.loc[4,'tdvp',1]

expectation approximation ratio               mean    5.556973e-01
                                              std     3.201487e-01
                                              min     1.338465e-01
                                              max     9.632252e-01
groundspace overlap                           mean    4.643440e-01
                                              std     3.731296e-01
                                              min     1.987056e-02
                                              max     9.623940e-01
groundspace sharpness                         mean    3.318648e-01
                                              std     1.645832e-01
                                              min     1.395554e-01
                                              max     4.977434e-01
approximation ratio                           mean    8.500000e-01
                                              std     2.760878e-01
                                              min     3.142857

In [63]:


def cohens_d(m1:float,m2:float,s1:float,s2:float,n1:int,n2:int)->float:
    return (m1-m2)/np.sqrt((n1-1)*s1**2+(n2-1)*s2**2)

def hedges_g(m1:float,m2:float,s1:float,s2:float,n1:int,n2:int)->float:
    s_star = np.sqrt(
        ((n1-1)*s1**2+(n2-1)*s2**2)/(n1+n2-2)
    )
    if s_star==0: 
        s_star=1e-10
    a = n1+n2-2
    J = gamma(a/2)/(np.sqrt(a/2)*gamma(a/2-0.5))
    return J*(m1-m2)/s_star

def effect_size_mapping(col,stats,index=hedges_g):
    n,p = col.name
    for meas, comp in col.index:
        tdvp_stats = stats.loc[n,'tdvp',p]
        comp_stats = stats.loc[n,comp,p]
        col[meas,comp] = index(
            m1= tdvp_stats[meas,"mean"], m2=comp_stats[meas,"mean"],
            s1=tdvp_stats[meas,"std"], s2=comp_stats[meas,"std"],
            n1=tdvp_stats['relevance',"count"], n2=comp_stats['relevance',"count"],
        )
        
def effect_size_categorize(g:float) -> Literal['negligible', 'small', 'medium', 'large']:
    if g<0.2:
        return "negligible"
    elif g<0.5:
        return "small"
    elif g<0.8:
        return "medium"
    else:
        return "large"

In [ ]:
stats.drop(columns=['relevance'],level=0, inplace=False).columns.levels[0]

Index(['approximation ratio', 'circuit count',
       'expectation approximation ratio',
       'expectation approximation ratio per circuits', 'gate count',
       'groundspace overlap', 'groundspace sharpness', 'num_fun_calls',
       'path length', 'relevance', 'steps'],
      dtype='object')

In [ ]:
stats.columns.levels[0][:-2]

Index(['approximation ratio', 'circuit count',
       'expectation approximation ratio',
       'expectation approximation ratio per circuits', 'gate count',
       'groundspace overlap', 'groundspace sharpness', 'num_fun_calls',
       'path length'],
      dtype='object')

In [ ]:
stats.xs("tdvp", level="Algorithm", drop_level=True).index

MultiIndex([(3, 1),
            (3, 2),
            (4, 1),
            (4, 2),
            (4, 3),
            (4, 4),
            (4, 5),
            (5, 1),
            (5, 2),
            (5, 3),
            (5, 4)],
           names=['n', 'p'])

In [20]:
effect_size = pd.DataFrame(
    index=stats.xs("tdvp", level="Algorithm", drop_level=True).index,
    columns=pd.MultiIndex.from_product(
        [
            stats.columns.levels[0][:-2],
            ["scipy", "gradient_descent"],
        ]
    ),
)
effect_size


approximation ratio                  circuit count                   \
                  scipy gradient_descent         scipy gradient_descent   
n p                                                                       
3 1                 NaN              NaN           NaN              NaN   
  2                 NaN              NaN           NaN              NaN   
4 1                 NaN              NaN           NaN              NaN   
  2                 NaN              NaN           NaN              NaN   
  3                 NaN              NaN           NaN              NaN   
  4                 NaN              NaN           NaN              NaN   
  5                 NaN              NaN           NaN              NaN   
5 1                 NaN              NaN           NaN              NaN   
  2                 NaN              NaN           NaN              NaN   
  3                 NaN              NaN           NaN              NaN   
  4                 NaN              NaN           NaN              NaN   

    expectation approximation ratio                   \
                              scipy gradient_descent   
n p                                                    
3 1                             NaN              NaN   
  2                             NaN              NaN   
4 1                             NaN              NaN   
  2                             NaN              NaN   
  3                             NaN              NaN   
  4                             NaN              NaN   
  5                             NaN              NaN   
5 1                             NaN              NaN   
  2                             NaN              NaN   
  3                             NaN              NaN   
  4                             NaN              NaN   

    expectation approximation ratio per circuits                  gate count  \
                                           scipy gradient_descent      scipy   
n p                                                                            
3 1                                          NaN              NaN        NaN   
  2                                          NaN              NaN        NaN   
4 1                                          NaN              NaN        NaN   
  2                                          NaN              NaN        NaN   
  3                                          NaN              NaN        NaN   
  4                                          NaN              NaN        NaN   
  5                                          NaN              NaN        NaN   
5 1                                          NaN              NaN        NaN   
  2                                          NaN              NaN        NaN   
  3                                          NaN              NaN        NaN   
  4                                          NaN              NaN        NaN   

                     groundspace overlap                   \
    gradient_descent               scipy gradient_descent   
n p                                                         
3 1              NaN                 NaN              NaN   
  2              NaN                 NaN              NaN   
4 1              NaN                 NaN              NaN   
  2              NaN                 NaN              NaN   
  3              NaN                 NaN              NaN   
  4              NaN                 NaN              NaN   
  5              NaN                 NaN              NaN   
5 1              NaN                 NaN              NaN   
  2              NaN                 NaN              NaN   
  3              NaN                 NaN              NaN   
  4              NaN                 NaN              NaN   

    groundspace sharpness                  num_fun_calls                   \
                    scipy gradient_descent         scipy gradient_descent   
n p                                   

In [35]:
effect_size.apply(effect_size_mapping, axis=1, args=(stats,));

In [39]:
hedges_g(1.1,1,0,0,10,10)

957646427.023719

In [26]:
stats['approximation ratio']

mean       std       min       max
n Algorithm        p                                        
3 tdvp             1  1.000000  0.000000  1.000000  1.000000
                   2  1.000000  0.000000  1.000000  1.000000
  scipy            1  1.000000  0.000000  1.000000  1.000000
                   2  1.000000  0.000000  1.000000  1.000000
  gradient_descent 1  0.333333  0.942809 -0.333333  1.000000
                   2  1.000000  0.000000  1.000000  1.000000
4 tdvp             1  0.850000  0.276088  0.314286  1.000000
                   2  1.000000  0.000000  1.000000  1.000000
                   3  1.000000  0.000000  1.000000  1.000000
                   4  1.000000  0.000000  1.000000  1.000000
                   5  1.000000  0.000000  1.000000  1.000000
  scipy            1  0.869048  0.232189  0.428571  1.000000
                   2  1.000000  0.000000  1.000000  1.000000
                   3  1.000000  0.000000  1.000000  1.000000
                   4  0.964286  0.087482  0.785714  1.000000
                   5  1.000000  0.000000  1.000000  1.000000
  gradient_descent 1  0.100000  0.761041 -1.285714  1.000000
                   2  0.067460  0.263595 -0.428571  0.357143
                   3  0.504762  0.919035 -1.285714  1.000000
                   4  0.134921  0.844573 -1.285714  1.000000
                   5  0.151587  0.755871 -0.928571  1.000000
5 tdvp             1  0.515238  0.445836 -0.190476  1.000000
                   2  0.923810  0.228571  0.314286  1.000000
                   3  1.000000  0.000000  1.000000  1.000000
                   4  1.000000  0.000000  1.000000  1.000000
  scipy            1  0.779592  0.260607  0.428571  1.000000
                   2  0.879592  0.200062  0.510204  1.000000
                   3  0.928571  0.154303  0.571429  1.000000
                   4  0.971429  0.060234  0.857143  1.000000
  gradient_descent 1 -0.231633  0.488527 -1.285714  0.314286
                   2  0.234127  0.731186 -1.285714  1.000000
                   3 -0.158503  0.751500 -1.285714  1.000000
                   4  0.171020  0.895167 -1.285714  1.000000

In [41]:
effect_size.drop(columns=['num_fun_calls'],level=0, inplace=True)

In [66]:
effect_size.rename(lambda x: x.replace("_"," "), axis=1, level=0, inplace=True)
effect_size.rename(lambda x: x.replace("gradient_descent","GD"), axis=1, level=1, inplace=True)
effect_size.rename(lambda x: x.replace("scipy","COBYLA"), axis=1, level=1, inplace=True)
effect_size.drop(index=[3], level=0, inplace=True)
effect_size_directed = effect_size.copy()
effect_size_undirected = effect_size.copy().apply(np.abs)
effect_size_category = effect_size_undirected.copy().apply(lambda x: x.apply(effect_size_categorize))

In [59]:
effect_size_directed

Approximation ratio           Circuit count             \
                 COBYLA        GD        COBYLA         GD   
n p                                                          
4 1           -0.068903  1.208933     13.466604  13.140384   
  2                 0.0  4.616655      1.317935   1.292338   
  3                 0.0  0.660701      0.851146   0.840156   
  4            0.466154  1.169564      5.057729   5.021144   
  5                 0.0  1.179602      1.709059   1.700069   
5 1           -0.693276   1.52937      4.100889     4.0591   
  2            0.197394  1.187657      2.056962   2.041357   
  3            0.567141  1.888697      4.461406   4.442341   
  4             0.55924  1.091809      1.061756   1.058404   

    Expectation approximation ratio            \
                             COBYLA        GD   
n p                                             
4 1                       -0.078062  1.790508   
  2                        0.579214   6.59108   
  3                        0.730455  3.025058   
  4                        0.857109  4.552017   
  5                         0.37858  4.622825   
5 1                        -0.48583  1.535451   
  2                         1.01513  4.126366   
  3                        0.872262  6.162096   
  4                       -0.011778  4.478561   

    Expectation approximation ratio per circuits           Gate count  \
                                          COBYLA        GD     COBYLA   
n p                                                                     
4 1                                    -5.955991 -0.908317   6.273362   
  2                                    -4.797657  0.170779   1.097066   
  3                                    -2.671111  -0.03776   0.900838   
  4                                   -21.157234  0.338789   4.317769   
  5                                    -2.091172  0.360929    1.92263   
5 1                                    -5.453887  -1.01474   2.750824   
  2                                     -14.1204 -0.440118   1.615595   
  3                                   -30.041762  0.301245   3.247997   
  4                                    -5.819317  0.022308   1.055591   

              Groundspace overlap            Groundspace sharpness            \
           GD              COBYLA         GD                COBYLA        GD   
n p                                                                            
4 1  6.019495            0.205933    1.18325             -0.080837  0.173569   
  2  1.089854            0.596743  18.442253             -0.463243  1.317026   
  3  0.898831            0.568261   5.375869             -0.396015 -0.174018   
  4  4.315232            0.783773   4.534492             -0.569897 -0.375818   
  5  1.922136            0.433454   6.400803             -0.352469 -0.599815   
5 1  2.665073           -0.156747   0.752066             -0.557466  0.588297   
  2  1.607239            1.091701   3.504858              0.011083  0.873125   
  3  3.243967            0.808675   6.080383              0.107252  1.351707   
  4   1.05508            0.119516   3.857116              0.527886  1.531015   

    Path length            
         COBYLA        GD  
n p                        
4 1    4.196836  1.591686  
  2    7.458731  7.132095  
  3     3.63329  4.053178  
  4    8.353581  7.575652  
  5    4.003324   4.15597  
5 1    9.405778   2.13748  
  2    3.944524  3.772738  
  3    5.627975  5.455648  
  4    6.888549  7.053959

In [90]:
effect_size_undirected

Approximation ratio           Circuit count             \
                 COBYLA        GD        COBYLA         GD   
n p                                                          
4 1            0.068903  1.208933     13.466604  13.140384   
  2                 0.0  4.616655      1.317935   1.292338   
  3                 0.0  0.660701      0.851146   0.840156   
  4            0.466154  1.169564      5.057729   5.021144   
  5                 0.0  1.179602      1.709059   1.700069   
5 1            0.693276   1.52937      4.100889     4.0591   
  2            0.197394  1.187657      2.056962   2.041357   
  3            0.567141  1.888697      4.461406   4.442341   
  4             0.55924  1.091809      1.061756   1.058404   

    Expectation approximation ratio            \
                             COBYLA        GD   
n p                                             
4 1                        0.078062  1.790508   
  2                        0.579214   6.59108   
  3                        0.730455  3.025058   
  4                        0.857109  4.552017   
  5                         0.37858  4.622825   
5 1                         0.48583  1.535451   
  2                         1.01513  4.126366   
  3                        0.872262  6.162096   
  4                        0.011778  4.478561   

    Expectation approximation ratio per circuits           Gate count  \
                                          COBYLA        GD     COBYLA   
n p                                                                     
4 1                                     5.955991  0.908317   6.273362   
  2                                     4.797657  0.170779   1.097066   
  3                                     2.671111   0.03776   0.900838   
  4                                    21.157234  0.338789   4.317769   
  5                                     2.091172  0.360929    1.92263   
5 1                                     5.453887   1.01474   2.750824   
  2                                      14.1204  0.440118   1.615595   
  3                                    30.041762  0.301245   3.247997   
  4                                     5.819317  0.022308   1.055591   

              Groundspace overlap            Groundspace sharpness            \
           GD              COBYLA         GD                COBYLA        GD   
n p                                                                            
4 1  6.019495            0.205933    1.18325              0.080837  0.173569   
  2  1.089854            0.596743  18.442253              0.463243  1.317026   
  3  0.898831            0.568261   5.375869              0.396015  0.174018   
  4  4.315232            0.783773   4.534492              0.569897  0.375818   
  5  1.922136            0.433454   6.400803              0.352469  0.599815   
5 1  2.665073            0.156747   0.752066              0.557466  0.588297   
  2  1.607239            1.091701   3.504858              0.011083  0.873125   
  3  3.243967            0.808675   6.080383              0.107252  1.351707   
  4   1.05508            0.119516   3.857116              0.527886  1.531015   

    Path length            
         COBYLA        GD  
n p                        
4 1    4.196836  1.591686  
  2    7.458731  7.132095  
  3     3.63329  4.053178  
  4    8.353581  7.575652  
  5    4.003324   4.15597  
5 1    9.405778   2.13748  
  2    3.944524  3.772738  
  3    5.627975  5.455648  
  4    6.888549  7.053959

In [74]:
effect_size_category

Approximation ratio         Circuit count         \
                 COBYLA      GD        COBYLA     GD   
n p                                                    
4 1          negligible   large         large  large   
  2          negligible   large         large  large   
  3          negligible  medium         large  large   
  4               small   large         large  large   
  5          negligible   large         large  large   
5 1              medium   large         large  large   
  2          negligible   large         large  large   
  3              medium   large         large  large   
  4              medium   large         large  large   

    Expectation approximation ratio         \
                             COBYLA     GD   
n p                                          
4 1                      negligible  large   
  2                          medium  large   
  3                          medium  large   
  4                           large  large   
  5                           small  large   
5 1                           small  large   
  2                           large  large   
  3                           large  large   
  4                      negligible  large   

    Expectation approximation ratio per circuits             Gate count  \
                                          COBYLA          GD     COBYLA   
n p                                                                       
4 1                                        large       large      large   
  2                                        large  negligible      large   
  3                                        large  negligible      large   
  4                                        large       small      large   
  5                                        large       small      large   
5 1                                        large       large      large   
  2                                        large       small      large   
  3                                        large       small      large   
  4                                        large  negligible      large   

           Groundspace overlap         Groundspace sharpness              \
        GD              COBYLA      GD                COBYLA          GD   
n p                                                                        
4 1  large               small   large            negligible  negligible   
  2  large              medium   large                 small       large   
  3  large              medium   large                 small  negligible   
  4  large              medium   large                medium       small   
  5  large               small   large                 small      medium   
5 1  large          negligible  medium                medium      medium   
  2  large               large   large            negligible       large   
  3  large               large   large            negligible       large   
  4  large          negligible   large                medium       large   

    Path length         
         COBYLA     GD  
n p                     
4 1       large  large  
  2       large  large  
  3       large  large  
  4       large  large  
  5       large  large  
5 1       large  large  
  2       large  large  
  3       large  large  
  4       large  large

In [89]:
"{:.2f}".format(-1.2342584765874692765982769827568914356713784650134876)

'-1.23'

In [91]:
effect_size_combined = effect_size_category.copy()
for idx in effect_size_combined.index:
    for meas, comp in effect_size_combined.loc[idx].index:
        effect_size_combined.loc[idx][meas,comp] = effect_size_category.loc[idx][meas,comp] + " (" + "{:.2f}".format(effect_size_directed.loc[idx][meas,comp])+")"
effect_size_combined

Approximation ratio                 Circuit count                 \
                 COBYLA             GD         COBYLA             GD   
n p                                                                    
4 1  negligible (-0.07)   large (1.21)  large (13.47)  large (13.14)   
  2   negligible (0.00)   large (4.62)   large (1.32)   large (1.29)   
  3   negligible (0.00)  medium (0.66)   large (0.85)   large (0.84)   
  4        small (0.47)   large (1.17)   large (5.06)   large (5.02)   
  5   negligible (0.00)   large (1.18)   large (1.71)   large (1.70)   
5 1      medium (-0.69)   large (1.53)   large (4.10)   large (4.06)   
  2   negligible (0.20)   large (1.19)   large (2.06)   large (2.04)   
  3       medium (0.57)   large (1.89)   large (4.46)   large (4.44)   
  4       medium (0.56)   large (1.09)   large (1.06)   large (1.06)   

    Expectation approximation ratio                \
                             COBYLA            GD   
n p                                                 
4 1              negligible (-0.08)  large (1.79)   
  2                   medium (0.58)  large (6.59)   
  3                   medium (0.73)  large (3.03)   
  4                    large (0.86)  large (4.55)   
  5                    small (0.38)  large (4.62)   
5 1                   small (-0.49)  large (1.54)   
  2                    large (1.02)  large (4.13)   
  3                    large (0.87)  large (6.16)   
  4              negligible (-0.01)  large (4.48)   

    Expectation approximation ratio per circuits                      \
                                          COBYLA                  GD   
n p                                                                    
4 1                                large (-5.96)       large (-0.91)   
  2                                large (-4.80)   negligible (0.17)   
  3                                large (-2.67)  negligible (-0.04)   
  4                               large (-21.16)        small (0.34)   
  5                                large (-2.09)        small (0.36)   
5 1                                large (-5.45)       large (-1.01)   
  2                               large (-14.12)       small (-0.44)   
  3                               large (-30.04)        small (0.30)   
  4                                large (-5.82)   negligible (0.02)   

       Gate count               Groundspace overlap                 \
           COBYLA            GD              COBYLA             GD   
n p                                                                  
4 1  large (6.27)  large (6.02)        small (0.21)   large (1.18)   
  2  large (1.10)  large (1.09)       medium (0.60)  large (18.44)   
  3  large (0.90)  large (0.90)       medium (0.57)   large (5.38)   
  4  large (4.32)  large (4.32)       medium (0.78)   large (4.53)   
  5  large (1.92)  large (1.92)        small (0.43)   large (6.40)   
5 1  large (2.75)  large (2.67)  negligible (-0.16)  medium (0.75)   
  2  large (1.62)  large (1.61)        large (1.09)   large (3.50)   
  3  large (3.25)  large (3.24)        large (0.81)   large (6.08)   
  4  large (1.06)  large (1.06)   negligible (0.12)   large (3.86)   

    Groundspace sharpness                       Path length                
                   COBYLA                  GD        COBYLA            GD  
n p                                                                        
4 1    negligible (-0.08)   negligible (0.17)  large (4.20)  large (1.59)  
  2         small (-0.46)        large (1.32)  large (7.46)  large (7.13)  
  3         small (-0.40)  negligible (-0.17)  large (3.63)  large (4.05)  
  4        medium (-0.57)       small (-0.38)  large (8.35)  large (7.58)  
  5         small (-0.35)      medium (-0.60)  large (4.00)  large (4.16)  
5 1        medium (-0.56)       medium (0.59)  large (9.41)  large (2.14)  
  2     negligible (0.01)        large (0.87)  large (3.94)  large (3.77)  
  3     negligible (0.11)        lar

In [95]:
meas_abbrv = {
    'Approximation ratio': 'M1',
    'Groundspace overlap': 'M2',
    'Groundspace sharpness': 'M3',
    'Expectation approximation ratio': 'M4',
    'Gate count': 'M5',
    'Circuit count': 'M6',
    'Expectation approximation ratio per circuits': 'M7',
    'Path length': 'M8',
              }
effect_size_combined.rename(meas_abbrv, axis=1, level=0, inplace=True)
effect_size_combined.sort_index(axis=1, inplace=True)

In [96]:
effect_size_combined

M1                                 M2                 \
                 COBYLA             GD              COBYLA             GD   
n p                                                                         
4 1  negligible (-0.07)   large (1.21)        small (0.21)   large (1.18)   
  2   negligible (0.00)   large (4.62)       medium (0.60)  large (18.44)   
  3   negligible (0.00)  medium (0.66)       medium (0.57)   large (5.38)   
  4        small (0.47)   large (1.17)       medium (0.78)   large (4.53)   
  5   negligible (0.00)   large (1.18)        small (0.43)   large (6.40)   
5 1      medium (-0.69)   large (1.53)  negligible (-0.16)  medium (0.75)   
  2   negligible (0.20)   large (1.19)        large (1.09)   large (3.50)   
  3       medium (0.57)   large (1.89)        large (0.81)   large (6.08)   
  4       medium (0.56)   large (1.09)   negligible (0.12)   large (3.86)   

                     M3                                      M4                \
                 COBYLA                  GD              COBYLA            GD   
n p                                                                             
4 1  negligible (-0.08)   negligible (0.17)  negligible (-0.08)  large (1.79)   
  2       small (-0.46)        large (1.32)       medium (0.58)  large (6.59)   
  3       small (-0.40)  negligible (-0.17)       medium (0.73)  large (3.03)   
  4      medium (-0.57)       small (-0.38)        large (0.86)  large (4.55)   
  5       small (-0.35)      medium (-0.60)        small (0.38)  large (4.62)   
5 1      medium (-0.56)       medium (0.59)       small (-0.49)  large (1.54)   
  2   negligible (0.01)        large (0.87)        large (1.02)  large (4.13)   
  3   negligible (0.11)        large (1.35)        large (0.87)  large (6.16)   
  4       medium (0.53)        large (1.53)  negligible (-0.01)  large (4.48)   

               M5                           M6                             M7  \
           COBYLA            GD         COBYLA             GD          COBYLA   
n p                                                                             
4 1  large (6.27)  large (6.02)  large (13.47)  large (13.14)   large (-5.96)   
  2  large (1.10)  large (1.09)   large (1.32)   large (1.29)   large (-4.80)   
  3  large (0.90)  large (0.90)   large (0.85)   large (0.84)   large (-2.67)   
  4  large (4.32)  large (4.32)   large (5.06)   large (5.02)  large (-21.16)   
  5  large (1.92)  large (1.92)   large (1.71)   large (1.70)   large (-2.09)   
5 1  large (2.75)  large (2.67)   large (4.10)   large (4.06)   large (-5.45)   
  2  large (1.62)  large (1.61)   large (2.06)   large (2.04)  large (-14.12)   
  3  large (3.25)  large (3.24)   large (4.46)   large (4.44)  large (-30.04)   
  4  large (1.06)  large (1.06)   large (1.06)   large (1.06)   large (-5.82)   

                                   M8                
                     GD        COBYLA            GD  
n p                                                  
4 1       large (-0.91)  large (4.20)  large (1.59)  
  2   negligible (0.17)  large (7.46)  large (7.13)  
  3  negligible (-0.04)  large (3.63)  large (4.05)  
  4        small (0.34)  large (8.35)  large (7.58)  
  5        small (0.36)  large (4.00)  large (4.16)  
5 1       large (-1.01)  large (9.41)  large (2.14)  
  2       small (-0.44)  large (3.94)  large (3.77)  
  3        small (0.30)  large (5.63)  large (5.46)  
  4   negligible (0.02)  large (6.89)  large (7.05)

In [114]:
effect_size_directed.astype(float).round(2).to_latex("results/effect_size_directed.tex")

In [116]:
stats.head(10)

expectation approximation ratio                      \
                                                mean       std       min   
n Algorithm        p                                                       
3 tdvp             1                        0.911088  0.073714  0.858965   
                   2                        0.930390  0.063845  0.885245   
  scipy            1                        0.778976  0.112656  0.699316   
                   2                        0.930466  0.058679  0.888974   
  gradient_descent 1                        0.306478  0.432672  0.000533   
                   2                        0.652124  0.047679  0.618410   
4 tdvp             1                        0.555697  0.320149  0.133846   
                   2                        0.914538  0.023949  0.873142   
                   3                        0.926812  0.027653  0.895857   
                   4                        0.943432  0.049538  0.869999   

                               groundspace overlap                      \
                           max                mean       std       min   
n Algorithm        p                                                     
3 tdvp             1  0.963212            0.869012  0.145009  0.766475   
                   2  0.975536            0.931829  0.071453  0.881304   
  scipy            1  0.858635            0.766332  0.006235  0.761923   
                   2  0.971958            0.893380  0.117401  0.810365   
  gradient_descent 1  0.612424            0.358776  0.307027  0.141675   
                   2  0.685838            0.636691  0.049783  0.601489   
4 tdvp             1  0.963225            0.464344  0.373130  0.019871   
                   2  0.947408            0.895209  0.049664  0.831052   
                   3  0.966206            0.866436  0.068250  0.795922   
                   4  0.974706            0.930693  0.062414  0.843536   

                               groundspace sharpness            ...   steps  \
                           max                  mean       std  ...     max   
n Algorithm        p                                            ...           
3 tdvp             1  0.971549              0.294665  0.181596  ...    87.0   
                   2  0.982354              0.227546  0.135601  ...   147.0   
  scipy            1  0.770741              0.423131  0.003925  ...    24.0   
                   2  0.976395              0.271914  0.169845  ...    50.0   
  gradient_descent 1  0.575877              0.421463  0.102879  ...    59.0   
                   2  0.671892              0.479558  0.014190  ...    95.0   
4 tdvp             1  0.962394              0.331865  0.164583  ...    82.0   
                   2  0.943499              0.297600  0.061860  ...   701.0   
                   3  0.967577              0.324697  0.090627  ...  3000.0   
                   4  0.977556              0.232359  0.100608  ...   225.0   

                     path length                                \
                            mean       std       min       max   
n Algorithm        p                                             
3 tdvp             1    0.912754  0.036438  0.886988  0.938520   
                   2    0.823661  0.014654  0.813299  0.834024   
  scipy            1    0.131371  0.094648  0.064445  0.198298   
                   2    0.081390  0.084589  0.021576  0.141203   
  gradient_descent 1    0.230496  0.310428  0.010990  0.450001   
                   2    0.088509  0.090446  0.024554  0.152464   
4 tdvp             1    0.837968  0.199199  0.434681  0.955152   
                   2    0.782392  0.116077  0.602306  0.896681   
                   3    0.593010  0.182262  0.372322  0.862888   
                   4    0.675338  0.099432  0.564874  0.806639   

                     expectation approximation ratio per circuits            \
                                                             mean       std   
n Algorithm        p    

In [118]:
stats_total = pd.DataFrame(index=stats.columns, columns=['tdvp', 'scipy', 'gradient_descent'])
stats_total

tdvp scipy gradient_descent
expectation approximation ratio              mean   NaN   NaN              NaN
                                             std    NaN   NaN              NaN
                                             min    NaN   NaN              NaN
                                             max    NaN   NaN              NaN
groundspace overlap                          mean   NaN   NaN              NaN
                                             std    NaN   NaN              NaN
                                             min    NaN   NaN              NaN
                                             max    NaN   NaN              NaN
groundspace sharpness                        mean   NaN   NaN              NaN
                                             std    NaN   NaN              NaN
                                             min    NaN   NaN              NaN
                                             max    NaN   NaN              NaN
approximation ratio                          mean   NaN   NaN              NaN
                                             std    NaN   NaN              NaN
                                             min    NaN   NaN              NaN
                                             max    NaN   NaN              NaN
circuit count                                mean   NaN   NaN              NaN
                                             std    NaN   NaN              NaN
                                             min    NaN   NaN              NaN
                                             max    NaN   NaN              NaN
gate count                                   mean   NaN   NaN              NaN
                                             std    NaN   NaN              NaN
                                             min    NaN   NaN              NaN
                                             max    NaN   NaN              NaN
num_fun_calls                                mean   NaN   NaN              NaN
                                             std    NaN   NaN              NaN
                                             min    NaN   NaN              NaN
                                             max    NaN   NaN              NaN
steps                                        mean   NaN   NaN              NaN
                                             std    NaN   NaN              NaN
                                             min    NaN   NaN              NaN
                                             max    NaN   NaN              NaN
path length                                  mean   NaN   NaN              NaN
                                             std    NaN   NaN              NaN
                                             min    NaN   NaN              NaN
                                             max    NaN   NaN              NaN
expectation approximation ratio per circuits mean   NaN   NaN              NaN
                                             std    NaN   NaN              NaN
relevance                                    count  NaN   NaN              NaN
                                             total  NaN   NaN              NaN
                                             ratio  NaN   NaN              NaN

In [144]:
tdvp_total = pd.DataFrame(index=data.index, columns=stats.columns.levels[0]).drop(columns=['expectation approximation ratio per circuits', 'relevance'])
scipy_total = pd.DataFrame(index=data.index, columns=stats.columns.levels[0]).drop(columns=['expectation approximation ratio per circuits', 'relevance'])
gd_total = pd.DataFrame(index=data.index, columns=stats.columns.levels[0]).drop(columns=['expectation approximation ratio per circuits', 'relevance'])
tdvp_total

approximation ratio circuit count expectation approximation ratio  \
n p i                                                                     
3 1 0                 NaN           NaN                             NaN   
    1                 NaN           NaN                             NaN   
  2 0                 NaN           NaN                             NaN   
    1                 NaN           NaN                             NaN   
4 1 0                 NaN           NaN                             NaN   
...                   ...           ...                             ...   
5 4 5                 NaN           NaN                             NaN   
    6                 NaN           NaN                             NaN   
    7                 NaN           NaN                             NaN   
    8                 NaN           NaN                             NaN   
    9                 NaN           NaN                             NaN   

      gate count groundspace overlap groundspace sharpness num_fun_calls  \
n p i                                                                      
3 1 0        NaN                 NaN                   NaN           NaN   
    1        NaN                 NaN                   NaN           NaN   
  2 0        NaN                 NaN                   NaN           NaN   
    1        NaN                 NaN                   NaN           NaN   
4 1 0        NaN                 NaN                   NaN           NaN   
...          ...                 ...                   ...           ...   
5 4 5        NaN                 NaN                   NaN           NaN   
    6        NaN                 NaN                   NaN           NaN   
    7        NaN                 NaN                   NaN           NaN   
    8        NaN                 NaN                   NaN           NaN   
    9        NaN                 NaN                   NaN           NaN   

      path length steps  
n p i                    
3 1 0         NaN   NaN  
    1         NaN   NaN  
  2 0         NaN   NaN  
    1         NaN   NaN  
4 1 0         NaN   NaN  
...           ...   ...  
5 4 5         NaN   NaN  
    6         NaN   NaN  
    7         NaN   NaN  
    8         NaN   NaN  
    9         NaN   NaN  

[74 rows x 9 columns]

In [170]:
data_total = pd.concat([tdvp_total, scipy_total, gd_total], keys=['tdvp', 'scipy', 'gradient_descent'], axis=1)
data_total

tdvp                                                \
      approximation ratio circuit count expectation approximation ratio   
n p i                                                                     
3 1 0                 1.0        2773.0                        0.858965   
    1                 1.0        3133.0                        0.963212   
  2 0                 1.0       10909.0                        0.885245   
    1                 1.0       15877.0                        0.975536   
4 1 0                 1.0        6697.0                        0.521938   
...                   ...           ...                             ...   
5 4 5                 NaN           NaN                             NaN   
    6                 NaN           NaN                             NaN   
    7                 1.0     1135001.0                        0.740394   
    8                 NaN           NaN                             NaN   
    9                 1.0      422501.0                        0.880197   

                                                                             \
         gate count groundspace overlap groundspace sharpness num_fun_calls   
n p i                                                                         
3 1 0      608040.0            0.766475              0.423073            56   
    1      863448.0            0.971549              0.166258            62   
  2 0     5531016.0            0.881304               0.32343            74   
    1     9946380.0            0.982354              0.131662           116   
4 1 0     1540540.0             0.45255              0.497743            44   
...             ...                 ...                   ...           ...   
5 4 5           NaN                 NaN                   NaN             0   
    6           NaN                 NaN                   NaN             0   
    7  2278223078.0             0.66672              0.471386           380   
    8           NaN                 NaN                   NaN             0   
    9   614273258.0            0.866424              0.340197           134   

                                      scipy  ...          gradient_descent  \
      path length steps approximation ratio  ... steps approximation ratio   
n p i                                        ...                             
3 1 0     0.93852    77                 1.0  ...    23                 1.0   
    1    0.886988    87                 1.0  ...    24           -0.333333   
  2 0    0.813299   101                 1.0  ...    48                 1.0   
    1    0.834024   147                 1.0  ...    50                 1.0   
4 1 0    0.898566    62                 1.0  ...    20                 1.0   
...           ...   ...                 ...  ...   ...                 ...   
5 4 5         NaN  None                 1.0  ...    99           -1.285714   
    6         NaN  None                 1.0  ...    72            0.571429   
    7    0.624708   454                 1.0  ...   119            0.510204   
    8         NaN  None                 1.0  ...    84           -1.285714   
    9    0.897946   169                 1.0  ...   170                 1.0   

                                                                \
      circuit count expectation approximation ratio gate count   
n p i                                                            
3 1 0           237                        0.612424      34032   
    1           237                        0.000533      42540   
  2 0           761                         0.61841      61614   
    1           761                        0.685838      75306   
4 1 0           221                        0.292312      77528   
...             ...                             ...        ...   
5 4 5          5025                       -0.167506    1695870   
    6          5025                       -0.113199    1570250   
    7          5025                        0

In [230]:
data_total['gradient_descent']

approximation ratio circuit count expectation approximation ratio  \
n p i                                                                     
3 1 0                 1.0           237                        0.612424   
    1           -0.333333           237                        0.000533   
  2 0                 1.0           761                         0.61841   
    1                 1.0           761                        0.685838   
4 1 0                 1.0           221                        0.292312   
...                   ...           ...                             ...   
5 4 5           -1.285714          5025                       -0.167506   
    6            0.571429          5025                       -0.113199   
    7            0.510204          5025                        0.242827   
    8           -1.285714          5025                       -0.425301   
    9                 1.0          5025                        0.150114   

      expectation approximation ratio per circuits gate count  \
n p i                                                           
3 1 0                                     0.002584      34032   
    1                                     0.000002      42540   
  2 0                                     0.000813      61614   
    1                                     0.000901      75306   
4 1 0                                     0.001323      77528   
...                                            ...        ...   
5 4 5                                    -0.000033    1695870   
    6                                    -0.000023    1570250   
    7                                     0.000048    1444630   
    8                                    -0.000085    1193390   
    9                                      0.00003    1067770   

      groundspace overlap groundspace sharpness num_fun_calls path length  \
n p i                                                                       
3 1 0            0.575877              0.494209          None     0.01099   
    1            0.141675              0.348716          None    0.450001   
  2 0            0.601489              0.489592          None    0.024554   
    1            0.671892              0.469524          None    0.152464   
4 1 0              0.2403              0.427265          None     0.32405   
...                   ...                   ...           ...         ...   
5 4 5            0.072309              0.258999          None    0.069476   
    6            0.023653              0.151964          None     0.06262   
    7            0.069378              0.254097          None    0.074261   
    8            0.074148              0.262011          None    0.045234   
    9            0.140606              0.347615          None     0.04561   

      steps  
n p i        
3 1 0    59  
    1    59  
  2 0    95  
    1    95  
4 1 0    55  
...     ...  
5 4 5   314  
    6   314  
    7   314  
    8   314  
    9   314  

[74 rows x 10 columns]

In [172]:
for alg in data_total.columns.levels[0]:
    ser = data_total[alg]['expectation approximation ratio']/data_total[alg]['circuit count']
    data_total[alg, 'expectation approximation ratio per circuits'] = ser

In [178]:
data_total.sort_index(axis=1, inplace=True)

In [189]:
with open("results/data_total.p",'rb') as file:
    data_total = pickle.load(file)

In [195]:
for meas in stats_total.index.levels[0]:
    for alg in stats_total.columns:
        if meas != 'relevance':
            stats_total.loc[meas, 'mean'][alg] = data_total[alg, meas].mean()
            stats_total.loc[meas, 'std'][alg] = data_total[alg, meas].std()
            if meas != 'expectation approximation ratio per circuits':
                stats_total.loc[meas, 'min'][alg] = data_total[alg, meas].min()
                stats_total.loc[meas, 'max'][alg] = data_total[alg, meas].max()
            
        
            

In [231]:
for alg in stats_total.columns:
    stats_total.loc['relevance', 'count'][alg] = len(data_total[data_total[alg,'gate count'].apply(lambda x: not np.isnan(x))])
    stats_total.loc['relevance', 'total'][alg] = len(data_total[alg,'gate count'])
    stats_total.loc['relevance', 'ratio'][alg] = stats_total.loc['relevance', 'count'][alg]/stats_total.loc['relevance', 'total'][alg]

tdvp
scipy
gradient_descent


In [229]:
len(data_total[data_total['gradient_descent','gate count'].apply(lambda x: not np.isnan(x))])

74

In [278]:
stats_total.rename({'gradient_descent':'gd'}, axis=1, inplace=True)

In [301]:
stats_total

tdvp  \
expectation approximation ratio              mean            0.766499   
                                             std             0.266474   
                                             min            -0.001885   
                                             max             0.999978   
groundspace overlap                          mean            0.688814   
                                             std             0.322811   
                                             min             0.001621   
                                             max             0.999983   
groundspace sharpness                        mean              0.3086   
                                             std             0.130104   
                                             min             0.004178   
                                             max             0.499365   
approximation ratio                          mean            0.892778   
                                             std             0.274183   
                                             min            -0.190476   
                                             max                  1.0   
circuit count                                mean       351638.566667   
                                             std        905807.622728   
                                             min               2773.0   
                                             max            6257501.0   
gate count                                   mean    567492444.233333   
                                             std    1684716397.003699   
                                             min             608040.0   
                                             max        11406151186.0   
num_fun_calls                                mean           195.27027   
                                             std           420.278391   
                                             min                    0   
                                             max                 2588   
steps                                        mean          290.333333   
                                             std           524.700937   
                                             min                   36   
                                             max                 3000   
path length                                  mean            0.742549   
                                             std             0.178936   
                                             min             0.325542   
                                             max             0.999191   
expectation approximation ratio per circuits mean            0.000027   
                                             std             0.000058   
relevance                                    count                 60   
                                             total                 74   
                                             ratio           0.810811   

                                                           scipy  \
expectation approximation ratio              mean         0.7323   
                                             std        0.223843   
                                             min        0.096577   
                                             max        0.995812   
groundspace overlap                          mean       0.591318   
                                             std        0.334467   
                                             min        0.000165   
                                             max         0.99674   
groundspace sharpness                        mean       0.335777   
                                             std        0.137158   
                                             min        0.012831   
                                             max         0.49996   
approximation ratio                          mean       0.926917   
                                             s

In [279]:
with open("results/stats_total.p",'wb') as file:
    pickle.dump(stats_total,file)

In [289]:
total_effect_size= pd.DataFrame(index=stats_total.index.levels[0], columns=['scipy', 'gd'])
total_effect_size.drop(['steps', 'num_fun_calls', 'relevance'],inplace=True)
total_effect_size

,scipy,gd
approximation ratio,NaN,NaN
circuit count,NaN,NaN
expectation approximation ratio,NaN,NaN
expectation approximation ratio per circuits,NaN,NaN
gate count,NaN,NaN
groundspace overlap,NaN,NaN
groundspace sharpness,NaN,NaN
path length,NaN,NaN


In [280]:
hedges_g(
    stats_total["tdvp"]["expectation approximation ratio"]['mean'],
    stats_total["gd"]["expectation approximation ratio"]['mean'],
    stats_total['tdvp']['expectation approximation ratio']['std'],stats_total['gd']['expectation approximation ratio']['std'],
    stats_total['tdvp']['relevance', 'count'],stats_total['gd']['relevance', 'count']
)


2.9719709009317032

In [299]:
for meas in total_effect_size.index:
    for alg in ["scipy", "gd"]:
        total_effect_size.loc[meas, alg] = hedges_g(
            m1=stats_total["tdvp"][meas]['mean'],
            m2=stats_total[alg][meas]['mean'],
            s1=stats_total['tdvp'][meas]['std'],
            s2=stats_total[alg][meas]['std'],
            n1=stats_total['tdvp']['relevance', 'count'],
            n2=stats_total[alg]['relevance', 'count']
        )


In [305]:
total_effect_size.apply(np.abs)

,scipy,gd
approximation ratio,0.154559,1.350597
circuit count,0.577227,0.573248
expectation approximation ratio,0.139464,2.971971
expectation approximation ratio per circuits,2.108684,0.309666
gate count,0.500962,0.500454
groundspace overlap,0.294375,2.323378
groundspace sharpness,0.201578,0.387002
path length,5.025245,3.077535


In [307]:
total_effect_size_category = total_effect_size.copy()
for idx in total_effect_size_category.index:
    for alg in total_effect_size_category.loc[idx].index:
        if total_effect_size_category.loc[idx, alg] < 0.2:
            total_effect_size_category.loc[idx, alg] = "negligible"
        elif total_effect_size_category.loc[idx, alg] < 0.8:
            total_effect_size_category.loc[idx, alg] = "small"
        elif total_effect_size_category.loc[idx, alg] < 1.2:
            total_effect_size_category.loc[idx, alg] = "medium"
        else:
            total_effect_size_category.loc[idx, alg] = "large"

In [312]:
total_effect_size_combined = total_effect_size_category.copy()
for meas in total_effect_size_combined.index:
    for comp in total_effect_size_combined.loc[idx].index:
        total_effect_size_combined.loc[meas][comp] = total_effect_size_category.loc[meas][comp] + " (" + "{:.2f}".format(total_effect_size.loc[meas][comp])+")"

In [316]:
total_effect_size_combined

,scipy,gd
approximation ratio,negligible (-0.15),large (1.35)
circuit count,small (0.58),small (0.57)
expectation approximation ratio,negligible (0.14),large (2.97)
expectation approximation ratio per circuits,negligible (-2.11),negligible (-0.31)
gate count,small (0.50),small (0.50)
groundspace overlap,small (0.29),large (2.32)
groundspace sharpness,negligible (-0.20),small (0.39)
path length,NA,large (3.08)


In [317]:
with open("results/total_effect_size_combined.p",'wb') as file:
    pickle.dump(total_effect_size_combined,file)

In [325]:
np.array(data_total['tdvp']['path length'])

array([0.9385196653061596, 0.886988498178056, 0.8132991915420401,
       0.834023546460368, 0.898566465021044, 0.9098216581203107,
       0.9551522874502899, 0.8884281807732192, 0.4346812504997844,
       0.9411590552586566, 0.6818773551482376, 0.8966807123533533,
       0.8592013964597753, 0.8577096240528604, 0.602306232922257,
       0.7965784688765338, 0.8628877880873481, 0.556574013390177,
       0.6571032231743528, 0.5161632105005212, 0.3723222468075711, nan,
       0.5648741229461829, 0.6642272985245559, 0.6656117225122345, nan,
       0.806639148451193, nan, nan, 0.3255419493157514,
       0.6943631641123666, nan, 0.5651577563331852, nan,
       0.9380418836840865, 0.8622280537077172, 0.8868133234266093,
       0.9991908287960958, 0.9987348932615168, 0.9873462055465452,
       0.8744911256946677, 0.7358668422213904, 0.8933445582257235,
       0.9855383134883111, 0.6877881897748995, 0.4990241933554088,
       0.5744335625775114, nan, 0.7272060983125394, 0.4811458890969179,
      

In [342]:
data_total.astype('float')

gradient_descent                                                \
      approximation ratio circuit count expectation approximation ratio   
n p i                                                                     
3 1 0            1.000000         237.0                        0.612424   
    1           -0.333333         237.0                        0.000533   
  2 0            1.000000         761.0                        0.618410   
    1            1.000000         761.0                        0.685838   
4 1 0            1.000000         221.0                        0.292312   
...                   ...           ...                             ...   
5 4 5           -1.285714        5025.0                       -0.167506   
    6            0.571429        5025.0                       -0.113199   
    7            0.510204        5025.0                        0.242827   
    8           -1.285714        5025.0                       -0.425301   
    9            1.000000        5025.0                        0.150114   

                                                               \
      expectation approximation ratio per circuits gate count   
n p i                                                           
3 1 0                                     0.002584    34032.0   
    1                                     0.000002    42540.0   
  2 0                                     0.000813    61614.0   
    1                                     0.000901    75306.0   
4 1 0                                     0.001323    77528.0   
...                                            ...        ...   
5 4 5                                    -0.000033  1695870.0   
    6                                    -0.000023  1570250.0   
    7                                     0.000048  1444630.0   
    8                                    -0.000085  1193390.0   
    9                                     0.000030  1067770.0   

                                                                           \
      groundspace overlap groundspace sharpness num_fun_calls path length   
n p i                                                                       
3 1 0            0.575877              0.494209           NaN    0.010990   
    1            0.141675              0.348716           NaN    0.450001   
  2 0            0.601489              0.489592           NaN    0.024554   
    1            0.671892              0.469524           NaN    0.152464   
4 1 0            0.240300              0.427265           NaN    0.324050   
...                   ...                   ...           ...         ...   
5 4 5            0.072309              0.258999           NaN    0.069476   
    6            0.023653              0.151964           NaN    0.062620   
    7            0.069378              0.254097           NaN    0.074261   
    8            0.074148              0.262011           NaN    0.045234   
    9            0.140606              0.347615           NaN    0.045610   

              ...                tdvp                \
       steps  ... approximation ratio circuit count   
n p i         ...                                     
3 1 0   59.0  ...                 1.0        2773.0   
    1   59.0  ...                 1.0        3133.0   
  2 0   95.0  ...                 1.0       10909.0   
    1   95.0  ...                 1.0       15877.0   
4 1 0   55.0  ...                 1.0        6697.0   
...      ...  ...                 ...           ...   
5 4 5  314.0  ...                 NaN           NaN   
    6  314.0  ...                 NaN           NaN   
    7  314.0  ...                 1.0     1135001.0   
    8  314.0  ...                 NaN           NaN   
    9  314.0  ...                 1.0      422501.0   

                                       \
      expectation approximation ratio   
n p i                                   
3 1 0                        0.858965   
    1                        0.96

In [337]:
data_total['tdvp','path length'].dropna() 

False

In [370]:
res = statistics.ttest_ind(data_total['gd','path length'].astype('float'), data_total['tdvp','path length'].dropna().astype('float'))

SyntaxError: incomplete input (1993234629.py, line 1)

In [355]:
data_total.rename({'gradient_descent':'gd'}, axis=1, inplace=True)

In [375]:
total_ttest = total_effect_size.copy()
for meas in total_effect_size.index:
    for alg in ["scipy", "gd"]:
        total_ttest.loc[meas, alg] =  statistics.ttest_ind(
            data_total['tdvp', meas].dropna().astype('float'),
            data_total[alg, meas].astype('float'),
        )[1]
total_ttest

,scipy,gd
approximation ratio,0.372538,0.0
circuit count,0.001083,0.001169
expectation approximation ratio,0.420896,0.0
expectation approximation ratio per circuits,0.0,0.075308
gate count,0.00437,0.004409
groundspace overlap,0.090697,0.0
groundspace sharpness,0.245324,0.026734
path length,0.0,0.0
